# Basic RNNs

In [ ]:
n_inputs = 3
n_neurons = 5

In [ ]:
import tensorflow as tf

In [ ]:
X0=tf.placeholder(tf.float32,[None, n_inputs])

In [ ]:
X1=tf.placeholder(tf.float32,[None, n_inputs])

In [ ]:
Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons], dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons], dtype=tf.float32))
b= tf.Variable(tf.zeros([1,n_neurons], dtype=tf.float32))

In [ ]:
Y0 = tf.tanh(tf.matmul(X0, Wx) +b)
Y1 = tf.tanh(tf.matmul(Y0, Wy) + tf.matmul(X1, Wx) +b)

In [ ]:
init=tf.global_variables_initializer()

In [ ]:
import numpy as np

In [ ]:
X0_batch =  np.array([[0,1,2],[3,4,5],[6,7,8],[9,0,1]])

In [ ]:
import pandas as pd
X = pd.DataFrame(X0_batch)

In [ ]:
X

In [ ]:
X1_batch =  np.array([[9,8,7],[0,0,0],[6,5,4],[3,2,1]])

In [ ]:
with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})

In [ ]:
print(Y0_val)

In [ ]:
print(Y1_val)

# Static Unrolling Through Time

In [ ]:
import tensorflow as tf

In [ ]:
n_inputs = 3
n_neurons = 5

In [ ]:
X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

In [ ]:
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)

In [ ]:
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell,[X0,X1], dtype=tf.float32)
Y0, Y1=output_seqs
#function returns two objects. first is a python list containing the output tensors for each time step. 
#The second is a tensor containing the final states of the network. When you are using basic cells, the final state is simply
#equal to last output.

In [ ]:
import numpy as np
X0_batch =  np.array([[0,1,2],[3,4,5],[6,7,8],[9,0,1]])
X1_batch =  np.array([[9,8,7],[0,0,0],[6,5,4],[3,2,1]])

In [ ]:
init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    Y0_val, Y1_val = sess.run([Y0,Y1],feed_dict={X0: X0_batch, X1:X1_batch })

In [ ]:
print(Y0_val)

In [ ]:
print(Y1_val)

In [ ]:
#If there were 50 time steps, it would not be very convenient to have to define 50 input placeholders and 50 
#output tensors. We can simplify this.

#it takes a single input placeholder of shape  [None, n_steps, n_inputs] where the first dimension is 
#the mini-batch size.

In [ ]:
n_steps = 2
n_inputs = 3
n_neurons = 5

In [ ]:
import tensorflow as tf
X=tf.placeholder(tf.float32, [None, n_steps, n_inputs])

In [ ]:
X_seqs = tf.unstack(tf.transpose(X, perm=[1,0,2]))

In [ ]:
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)

In [ ]:
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell,X_seqs, dtype=tf.float32)

In [ ]:
outputs = tf.transpose(tf.stack(output_seqs), perm=[1,0,2])

In [ ]:
import numpy as np
X_batch = np.array([ 
    [[0,1,2],[9,8,7]],#instance 0
    [[3,4,5],[0,0,0]],#instance 1
    [[6,7,8],[6,5,4]],#instance 2
    [[9,0,1],[3,2,1]]])#instance 3

In [ ]:
init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    init.run()
    outputs_val = outputs.eval(feed_dict={X:X_batch})

In [ ]:
print(outputs_val)

# Dynamic Unrolling Through Time

In [ ]:
n_steps = 2
n_inputs = 3
n_neurons = 5

In [ ]:
import tensorflow as tf
X=tf.placeholder(tf.float32, [None, n_steps, n_inputs])

In [ ]:
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)

In [ ]:
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

In [ ]:
import numpy as np
X_batch = np.array([ 
    [[0,1,2],[9,8,7]],#instance 0
    [[3,4,5],[0,0,0]],#instance 1
    [[6,7,8],[6,5,4]],#instance 2
    [[9,0,1],[3,2,1]]])#instance 3

In [ ]:
init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    init.run()
    outputs_val = outputs.eval(feed_dict={X:X_batch})

In [ ]:
print(outputs_val)

# Handling Variable Length Input Sequences

In [ ]:
import tensorflow as tf
n_neurons = 5
n_inputs = 3
n_steps =2
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
seq_length = tf.placeholder(tf.int32, [None])
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units = n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32, sequence_length= seq_length)

In [ ]:
import numpy as np
X_batch = np.array([
    [[0,1,2],[9,8,7]], #instance 0 
    [[3,4,5],[0,0,0]], #instance 1
    [[6,7,8],[6,5,4]], #instance 2
    [[9,0,1],[3,2,1]], #instance 3
])

seq_length_batch = np.array([2,1,2,2])

In [ ]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    outputs_val, states_val = sess.run([outputs, states], feed_dict={X: X_batch, seq_length:seq_length_batch})

In [ ]:
print(outputs_val)

In [ ]:
print(states_val)
#Second row is only for t=0

# Training a Sequence Classifier

In [ ]:
n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10

learning_rate =0.001

In [ ]:
import tensorflow as tf

In [ ]:
X =  tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int64, [None])

In [ ]:
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

#dimensionality of states is batch_size X n_neurons

In [ ]:
#tf.layers.dense adds a single layer to your network. The second argument is the number of neurons/nodes of the layer
logits = tf.layers.dense(inputs = states, units = n_outputs)
#Output tensor the same shape as inputs except the last dimension is of size units.

In [ ]:
entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = logits)

In [ ]:
loss = tf.reduce_mean(entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
training = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits,y,1)
#correct = tf.equal(tf.argmax(logits, 1), y)
accuracy =tf.reduce_mean(tf.cast(correct,tf.float32))

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
mnist = input_data.read_data_sets("tmp/data/")

In [ ]:
X_test = mnist.test.images.reshape((-1, n_steps, n_inputs))
y_test = mnist.test.labels

In [ ]:
print(X_test.shape)
print(y_test.shape)

In [ ]:
n_epochs = 10
batch_size =50

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for epoch in range(n_epochs):
        for iterations in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            X_batch = X_batch.reshape((-1, n_steps, n_inputs))
            sess.run(training, feed_dict={X: X_batch, y: y_batch})
            states_val = states.eval(feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "Train Accuracy: ", acc_train, "Test Accuracy: ", acc_test)

In [ ]:
states_val.shape

In [ ]:
import tensorflow as tf
from tensorflow.contrib import rnn

#import mnist dataset
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("/tmp/data/",one_hot=True)

#define constants
#unrolled through 28 time steps
time_steps=28
#hidden LSTM units
num_units=128
#rows of 28 pixels
n_input=28
#learning rate for adam
learning_rate=0.001
#mnist is meant to be classified in 10 classes(0-9).
n_classes=10
#size of batch
batch_size=128


#Lets now declare placeholders and weights and bias variables which will be used to convert the output of shape 
#[batch_size,num_units] to [batch_size,n_classes] so that correct class can be predicted.

#weights and biases of appropriate shape to accomplish above task
out_weights=tf.Variable(tf.random_normal([num_units,n_classes]))
out_bias=tf.Variable(tf.random_normal([n_classes]))

#defining placeholders
#input image placeholder
x=tf.placeholder("float",[None,time_steps,n_input])
#input label placeholder
y=tf.placeholder("float",[None,n_classes])

#Now that we are receiving inputs of shape [batch_size,time_steps,n_input],we need to convert it into a list of 
#tensors of shape [batch_size,n_inputs] of length time_steps so that it can be then fed to static_rnn.

#processing the input tensor from [batch_size,n_steps,n_input] to "time_steps" number of [batch_size,n_input] tensors
input=tf.unstack(x ,time_steps,1)

lstm_layer=rnn.BasicLSTMCell(num_units,forget_bias=1)
outputs,_=rnn.static_rnn(lstm_layer,input,dtype="float32")

#converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
prediction=tf.matmul(outputs[-1],out_weights)+out_bias

loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
#optimization
opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

#model evaluation
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

#initialize variables
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    iter=1
    while iter<800:
        batch_x,batch_y=mnist.train.next_batch(batch_size=batch_size)

        batch_x=batch_x.reshape((batch_size,time_steps,n_input))

        sess.run(opt, feed_dict={x: batch_x, y: batch_y})

        if iter %10==0:
            acc=sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
            los=sess.run(loss,feed_dict={x:batch_x,y:batch_y})
            print("For iter ",iter)
            print("Accuracy ",acc)
            print("Loss ",los)
            print("__________________")

        iter=iter+1
    #calculating test accuracy
    test_data = mnist.test.images[:128].reshape((-1, time_steps, n_input))
    test_label = mnist.test.labels[:128]
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: test_data, y: test_label}))

# Training to predict Time Series

In [ ]:
n_steps = 20
n_neurons =100
n_inputs = 1
n_outputs = 1

In [ ]:
t_min, t_max  = 0, 30
resolution = 0.1

def time_series(t):
    return t * np.sin(t) / 3 + 2 * np.sin(t*5)

def next_batch(batch_size, n_steps):
    t0 = np.random.rand(batch_size, 1) * (t_max - t_min - n_steps * resolution)
    Ts = t0 + np.arange(0., n_steps + 1) * resolution
    ys = time_series(Ts)
    return ys[:, :-1].reshape(-1, n_steps, 1), ys[:, 1:].reshape(-1, n_steps, 1)

In [ ]:
import numpy as np
t = np.linspace(t_min, t_max, int((t_max - t_min) / resolution))

n_steps = 20
t_instance = np.linspace(12.2, 12.2 + resolution * (n_steps + 1), n_steps + 1)

In [ ]:
X_batch, y_batch = next_batch(1, n_steps)

In [ ]:
y_batch

In [ ]:
import tensorflow as tf

In [ ]:
X= tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])
cell = tf.contrib.rnn.OutputProjectionWrapper(tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu), output_size=n_outputs)
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

In [ ]:
learning_rate = 0.001

loss = tf.reduce_mean(tf.square(outputs - y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training = optimizer.minimize(loss)

In [ ]:
saver = tf.train.Saver()
n_iterations = 1500
batch_size = 50

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for iteration in range(n_iterations):
        X_batch, y_batch = next_batch(batch_size, n_steps)
        sess.run(training, feed_dict={X:X_batch, y:y_batch})
        if iteration % 100 == 0:
            mse = loss.eval(feed_dict={X:X_batch, y:y_batch})
            print(iteration, "\tMSE", mse)
    saver.save(sess, "./my_time_series_model")

In [ ]:
with tf.Session() as sess:                          # not shown in the book
    saver.restore(sess, "./my_time_series_model")   # not shown

    X_new = time_series(np.array(t_instance[:-1].reshape(-1, n_steps, n_inputs)))
    y_pred = sess.run(outputs, feed_dict={X: X_new})

In [ ]:
print(y_pred)

# Without using an OutputProjectionWrapper¶


In [ ]:
import tensorflow as tf 

n_steps = 20
n_inputs = 1
n_neurons = 100

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])

cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu)
rnn_outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

n_outputs = 1
learning_rate = 0.001

In [ ]:
stacked_rnn_outputs = tf.reshape(rnn_outputs, [-1, n_neurons])
stacked_outputs = tf.layers.dense(stacked_rnn_outputs, n_outputs)
outputs = tf.reshape(stacked_outputs, [-1, n_steps, n_outputs])

In [ ]:
loss = tf.reduce_mean(tf.square(outputs - y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [ ]:

n_iterationsn_iterat  = 1500
batch_size = 50

with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        X_batch, y_batch = next_batch(batch_size, n_steps)
        sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if iteration % 100 == 0:
            mse = loss.eval(feed_dict={X: X_batch, y: y_batch})
            print(iteration, "\tMSE:", mse)
    
    X_new = time_series(np.array(t_instance[:-1].reshape(-1, n_steps, n_inputs)))
    y_pred = sess.run(outputs, feed_dict={X: X_new})
    
    saver.save(sess, "./my_time_series_model")

In [ ]:
y_pred

# Deep RNNs

# MultiRNNCell

In [ ]:
n_inputs = 2
n_steps = 5

import tensorflow as tf 
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])

In [ ]:
n_neurons = 100
n_layers = 3

import numpy as np
layers = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
          for layer in range(n_layers)]
multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers)
outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)

In [ ]:
init = tf.global_variables_initializer()

In [ ]:
X_batch = np.random.rand(2, n_steps, n_inputs)

In [ ]:
X_batch

In [ ]:
with  tf.Session() as sess:
         init.run()
         outputs_val, states_val = sess.run([outputs, states], feed_dict={X: X_batch})

In [ ]:
#outputs_val[0] #first instance (batch =0)
#outputs_val[0][0] - outputs_val[0][4] # first instance and 5 time steps
#outputs_val[0][0][0] - outputs_val[0][0][99] # first instance and first time step and 100 neurons

In [ ]:
outputs_val.shape
#2 instances, 5 times steps, 100 neurons

In [ ]:
#states_val[0] - states_val[2] #number of layers
#states_val[0][0] - states_val[0][1]   #first layer, 2 instances

#states_val[0][0][0] - states_val[0][0][99] #first layer first instance and 100 neurons
#final states of each cell

# Train a LSTM to predict the next word using a sample short story, Aesop’s Fables (DEEP RNN)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import random
import collections
import time


In [ ]:
start_time = time.time()
def elapsed(sec):
    if sec<60:
        return str(sec) + " sec"
    elif sec<(60*60):
        return str(sec/60) + " min"
    else:
        return str(sec/(60*60)) + " hr"

In [ ]:
# Target log path
logs_path = '/Users/mustafamuratarat/Desktop'
writer = tf.summary.FileWriter(logs_path)

# Text file containing words for training
training_file = '/Users/mustafamuratarat/Desktop/belling_the_cat.txt'

In [ ]:
def read_data(fname):
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    content = [word for i in range(len(content)) for word in content[i].split()]
    content = np.array(content)
    return content

In [ ]:
training_data = read_data(training_file)
print("Loaded training data...")

In [ ]:
training_data 

In [ ]:
def build_dataset(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

In [ ]:
dictionary, reverse_dictionary = build_dataset(training_data)
vocab_size = len(dictionary)

In [ ]:
dictionary

In [ ]:
vocab_size

In [ ]:
#We are feeding 3 words at each time steps
n_input = 3

# number of units in RNN cell (basically n_neurons)
n_hidden = 512

display_step = 1000

# Parameters
learning_rate = 0.001
training_iters = 50000

# tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocab_size])

# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size]))
}

In [ ]:
def RNN(x, weights, biases):

    # reshape to [1, n_input]
    x = tf.reshape(x, [-1, n_input])

    # Generate a n_input-element sequence of inputs
    # (eg. [had] [a] [general] -> [20] [6] [33])
    x = tf.split(x,n_input,1)

    # 2-layer LSTM, each layer has n_hidden units.
    # Average Accuracy= 95.20% at 50k iter
    rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])

    # 1-layer LSTM with n_hidden units but with lower accuracy.
    # Average Accuracy= 90.60% 50k iter
    # Uncomment line below to test but comment out the 2-layer rnn.MultiRNNCell above
    # rnn_cell = rnn.BasicLSTMCell(n_hidden)

    # generate prediction
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

In [ ]:
# Launch the graph
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0

    writer.add_graph(session.graph)

    while step < training_iters:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_input+1)
#In the training process, at each step, 3 symbols are retrieved from the training data. 
#These 3 symbols are converted to integers to form the input vector.
        symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
        symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])

#The training label is a one-hot vector coming from the symbol after the 3 input symbols.
        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[dictionary[str(training_data[offset+n_input])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [training_data[i] for i in range(offset, offset + n_input)]
            symbols_out = training_data[offset + n_input]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))
        step += 1
        offset += (n_input+1)
    print("Optimization Finished!")
    print("Elapsed time: ", elapsed(time.time() - start_time))

# Applying Dropout

In [ ]:
t_min, t_max  = 0, 30
resolution = 0.1

def time_series(t):
    return t * np.sin(t) / 3 + 2 * np.sin(t*5)

def next_batch(batch_size, n_steps):
    t0 = np.random.rand(batch_size, 1) * (t_max - t_min - n_steps * resolution)
    Ts = t0 + np.arange(0., n_steps + 1) * resolution
    ys = time_series(Ts)
    return ys[:, :-1].reshape(-1, n_steps, 1), ys[:, 1:].reshape(-1, n_steps, 1)

import numpy as np
t = np.linspace(t_min, t_max, int((t_max - t_min) / resolution))

n_steps = 20
t_instance = np.linspace(12.2, 12.2 + resolution * (n_steps + 1), n_steps + 1)

In [ ]:
n_inputs = 1
n_neurons = 100
n_layers = 3
n_steps = 20
n_outputs = 1

In [ ]:
import tensorflow as tf
X  = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])

In [ ]:
keep_prob = tf.placeholder_with_default(1.0, shape=())
cells = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
         for layer in range(n_layers)]
cells_drop = [tf.contrib.rnn.DropoutWrapper(cell, input_keep_prob=keep_prob)
              for cell in cells]
multi_layer_cell = tf.contrib.rnn.MultiRNNCell(cells_drop)
rnn_outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)

In [ ]:
learning_rate = 0.01

stacked_rnn_outputs = tf.reshape(rnn_outputs, [-1, n_neurons])
stacked_outputs = tf.layers.dense(stacked_rnn_outputs, n_outputs)
outputs = tf.reshape(stacked_outputs, [-1, n_steps, n_outputs])

loss = tf.reduce_mean(tf.square(outputs - y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [ ]:
n_iterations = 1500
batch_size = 50
train_keep_prob = 0.5

with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        X_batch, y_batch = next_batch(batch_size, n_steps)
        _, mse = sess.run([training_op, loss],
                          feed_dict={X: X_batch, y: y_batch,
                                     keep_prob: train_keep_prob})
        if iteration % 100 == 0:                 
            print(iteration, "Training MSE:", mse) 
    
    saver.save(sess, "./my_dropout_time_series_model")

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, "./my_dropout_time_series_model")

    X_new = time_series(np.array(t_instance[:-1].reshape(-1, n_steps, n_inputs)))
    y_pred = sess.run(outputs, feed_dict={X: X_new})

In [ ]:
y_pred #20 time-steps

In [ ]:
# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

plt.title("Testing the model", fontsize=14)
plt.plot(t_instance[:-1], time_series(t_instance[:-1]), "bo", markersize=10, label="instance")
plt.plot(t_instance[1:], time_series(t_instance[1:]), "w*", markersize=10, label="target")
plt.plot(t_instance[1:], y_pred[0,:,0], "r.", markersize=10, label="prediction")
plt.legend(loc="upper left")
plt.xlabel("Time")

plt.show()

# Long Short Term Memory

In [ ]:
n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10
n_layers = 3

In [ ]:
learning_rate = 0.001

import tensorflow as tf
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])

In [ ]:
lstm_cells = [tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons)
              for layer in range(n_layers)]
multi_cell = tf.contrib.rnn.MultiRNNCell(lstm_cells)
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)
top_layer_h_state = states[-1][1]
#top_layer_h_state is a tuple containing 3 LSTM laters, each with dimension batch_size X n_neurons
logits = tf.layers.dense(top_layer_h_state, n_outputs, name="softmax")
#the shape of logits is batch_size X n_outputs
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy, name="loss")
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()

In [ ]:
states

In [ ]:
top_layer_h_state

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("tmp/data/")
X_test = mnist.test.images.reshape((-1, n_steps, n_inputs))
y_test = mnist.test.labels

In [ ]:
n_epochs = 10
batch_size = 150

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            X_batch = X_batch.reshape((batch_size, n_steps, n_inputs))
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print("Epoch", epoch, "Train accuracy =", acc_train, "Test accuracy =", acc_test)

# Binary String LSTM Example

In [1]:
import numpy as np
from random import shuffle
 
train_input = ['{0:020b}'.format(i) for i in range(2**20)]
shuffle(train_input)
train_input = [map(int,i) for i in train_input]
ti  = []
for i in train_input:
    temp_list = []
    for j in i:
            temp_list.append([j])
    ti.append(np.array(temp_list))
train_input = ti

In [2]:
train_output = []
 
for i in train_input:
    count = 0
    for j in i:
        if j[0] == 1:
            count+=1
    temp_list = ([0]*21)
    temp_list[count]=1
    train_output.append(temp_list)

In [3]:
NUM_EXAMPLES = 10000
test_input = train_input[NUM_EXAMPLES:]
test_output = train_output[NUM_EXAMPLES:] #everything beyond 10,000
 
train_input = train_input[:NUM_EXAMPLES]
train_output = train_output[:NUM_EXAMPLES] #till 10,000

#output is one-hot encoded

In [4]:
n_inputs = train_input[0][0].shape[0] #1
n_outputs = len(train_output[0]) #21
n_steps = train_input[0].shape[0] #20
n_neurons = 24

In [5]:
import tensorflow as tf
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_outputs])

/Users/mustafamuratarat/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
cell = tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons, state_is_tuple=True, forget_bias=1.0)
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

In [7]:
outputs= tf.transpose(outputs, [1, 0, 2])
last = tf.gather(outputs, int(outputs.get_shape()[0]) - 1)

In [14]:
learning_rate= 0.01

weights = {
    'out': tf.Variable(tf.truncated_normal([n_neurons, int(y.get_shape()[1])]))
}
biases = {
    'out': tf.Variable(tf.constant(0.1, shape=[y.get_shape()[1]]))
}

prediction = tf.matmul(last, weights['out'])+biases['out']
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
#optimization
opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

#model evaluation
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

#mistakes = tf.not_equal(tf.argmax(y, 1), tf.argmax(prediction, 1))
#error = tf.reduce_mean(tf.cast(mistakes, tf.float32))

batch_size = 1000
no_of_batches = int((len(train_input)) / batch_size)
#initialize variables
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    iter=1
    while iter<800:
        ptr = 0
        for j in range(no_of_batches):
            inp, out = train_input[ptr:ptr+batch_size], train_output[ptr:ptr+batch_size]
            ptr+=batch_size
            sess.run(opt,feed_dict={X: inp, y: out})
            if iter %10==0:
                acc=sess.run(accuracy,feed_dict={X: inp, y: out})
                los=sess.run(loss,feed_dict={X: inp, y: out})
                print("For iter ",iter)
                print("Accuracy ",acc)
                print("Loss ",los)
                print("__________________")
            iter=iter+1
        #calculating test accuracy
    print("Test Accuracy: ", sess.run(accuracy,{X:test_input, y:test_output }))

/Users/mustafamuratarat/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:97: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


For iter  10
Accuracy  0.135
Loss  2.2998965
__________________
For iter  20
Accuracy  0.126
Loss  2.125113
__________________
For iter  30
Accuracy  0.273
Loss  1.8046685
__________________
For iter  40
Accuracy  0.327
Loss  1.6183898
__________________
For iter  50
Accuracy  0.356
Loss  1.4043357
__________________
For iter  60
Accuracy  0.52
Loss  1.2267373
__________________
For iter  70
Accuracy  0.593
Loss  1.0665945
__________________
For iter  80
Accuracy  0.582
Loss  1.0659677
__________________
For iter  90
Accuracy  0.638
Loss  0.94609314
__________________
For iter  100
Accuracy  0.732
Loss  0.8160369
__________________
For iter  110
Accuracy  0.76
Loss  0.7647365
__________________
For iter  120
Accuracy  0.824
Loss  0.6959116
__________________
For iter  130
Accuracy  0.785
Loss  0.69355613
__________________
For iter  140
Accuracy  0.852
Loss  0.6123248
__________________
For iter  150
Accuracy  0.84
Loss  0.57954353
__________________
For iter  160
Accuracy  0.893
Loss 